# Linking named entities
Filip Gregora

In [1]:
import pandas as pd

data = pd.read_csv("data/NER_entities.csv")

In [2]:
data.head()

,label,text
0,symptom,jemný fibrózní proužek
1,procedura,neoadjuvantní CHT
2,medikace,Novalgin
3,symptom,Označena SLU v levé axile.
4,procedura,st.p. totální ME + SNB vlevo


In [61]:
symptom = data[data.label == "symptom"].drop(["label"], axis=1)
procedura = data[data.label == "procedura"].drop(["label"], axis=1)
medikace = data[data.label == "medikace"].drop(["label"], axis=1)
ne_symptom = data[data.label == "NE symptom"].drop(["label"], axis=1)
os_anamneza = data[data.label == "osobní anamnéza"].drop(["label"], axis=1)
ne_os_anamneza = data[data.label == "NE osobní anamnéza"].drop(["label"], axis=1)
ne_medikace = data[data.label == "NE medikace"].drop(["label"], axis=1)

In [4]:
assert(len(os_anamneza) + len(symptom) + len(procedura) + len(medikace) + len(ne_symptom) + len(ne_os_anamneza) + len(ne_medikace) \
            == len(data))

In [5]:
symptom.head(10)

,text
0,jemný fibrózní proužek
3,Označena SLU v levé axile.
18,mamma l.sin. dysplazie
24,další ojedinělé
29,stále brnění HKK i DKK
36,hmatná tu léze v DKK
59,další velikostní regrese TU
60,nespavost
64,vlevo jizva ZHK změklá
85,bolesti patní ostruhy vlevo


In [79]:
def clean_db(db):
    db["text"] = db["text"].apply(lambda x: x.strip(" ,.:").lower())
    db["text"] = db["text"].drop_duplicates()
    db.dropna(inplace=True)

In [7]:
symptom["text"] = clean_db(symptom["text"])
symptom.head(10)

,text
0,jemný fibrózní proužek
3,označena slu v levé axile
18,mamma l.sin. dysplazie
24,další ojedinělé
29,stále brnění hkk i dkk
36,hmatná tu léze v dkk
59,další velikostní regrese tu
60,nespavost
64,vlevo jizva zhk změklá
85,bolesti patní ostruhy vlevo


## Linking to MASH
Mash is international medical databaze: https://uts.nlm.nih.gov/uts/.

I tried search whole medical text in database. If it didn't success then I searched all combinations of words from text of length -1. I repeat it until I succeed (or all combinations have been tryied).

In [82]:
import requests
from bs4 import BeautifulSoup
import json
from itertools import combinations

# Do not search in databaze if it number or it is too short (shorter than 2)
def filter_short(string):
    return len(string) < 2 or string.isdigit()

def mash_search(string):
    splitted_input = (string.split(" "))
    result = []
    for j in range(len(splitted_input), 0, -1):
        for string in combinations(splitted_input, j): 
            if filter_short(" ".join(string)):
                continue
                
            path = 'https://uts-ws.nlm.nih.gov/rest/search/current'
            query = {
                     'string': " ".join(string),
                     'apiKey':'6a290909-c0d8-4db9-b531-7387929b334e',
            }
            res = requests.get(path, params=query)

            if res.status_code <= 200:
                data = json.loads(res.text)
                for j in data["result"]["results"]:
                    result.append((j["ui"], j["name"]))
            else:
                print(res.status_code, res.text)
        
        if len(result) != 0:
            break
                        
    return result
        
r = mash_search('bolesti patní ostruhy vlevo')
print(r)
# print(json.dumps(r, indent=4))

[('C0149756', 'Fasciitis, Plantar')]


In [85]:
clean_db(ne_medikace)
ne_medikace["search"] = ne_medikace["text"].apply(mash_search)
ne_medikace

,text,search
53,antikoncepce: 0,"[(C0700589, Contraceptive methods), (C0029151,..."
186,antikoncepce,"[(C0700589, Contraceptive methods), (C0029151,..."
557,tramal,"[(C0040611, Tramal), (C1699769, Tramal SR)]"
615,indometacin,"[(C3653099, indometacin, combinations), (C0021..."
856,hormonální léčba,"[(C0282402, Hormone replacement therapy), (C16..."
1306,hormonální léčba: 0,"[(C0282402, Hormone replacement therapy), (C16..."
1544,tamoxifen kontraindikován,"[(C0039286, tamoxifen), (C0079589, tamoxifen c..."
1620,ajatin,"[(C0101974, Ajatin)]"
1817,snesla chenmotepaii bez vomitu,"[(C0331055, Sambucus), (C0237154, Homelessness..."
2036,akt již vysazeno,"[(C0164786, Proto-Oncogene Proteins c-akt), (C..."


In [87]:
print(ne_medikace["search"][53])

[('C0700589', 'Contraceptive methods'), ('C0029151', 'Oral contraception'), ('C0419527', 'Contraception, Postcoital'), ('C0851160', 'Morning after pill method of contraception'), ('C0009866', 'Contraceptive Failure'), ('C0558252', 'Emergency Contraception'), ('C0086580', 'Male Contraception'), ('C0009892', 'Contraceptive History'), ('C0009899', 'Contraceptive Usage'), ('C2985296', 'Hormonal Contraception'), ('C0009870', 'Contraception, Immunologic'), ('C1168146', 'Transdermal contraception'), ('C1720756', 'Emergency Contraceptives'), ('C0086286', 'Female Contraception'), ('C4761168', 'Spermicidal contraceptive method'), ('C4761165', 'Intrauterine contraceptive method'), ('C0004764', 'Contraception, Barrier'), ('C0009891', 'Contraceptive Distribution'), ('C0009885', 'Contraceptive Availability'), ('C0009862', 'Contraception Behavior'), ('C4505082', 'Contraceptive Effectiveness'), ('C0021064', 'Immunological Fertility Control'), ('C4324371', 'Contraception via partner'), ('C1262153', 'In

In [ ]:
import xml.etree.ElementTree as elt

content = elt.parse('databaze/MeSH2023_Marc21.xml').getroot()

In [ ]:
for child in content:
#     print(child)
    pass
    